# Projekt 2

In [ ]:
import torch
import numpy as np
import time
import math
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as data
import pandas as pd

In [92]:
device = torch.device("cuda") 
device

device(type='cuda')

In [ ]:
if torch.cuda.is_available(): 
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)
    
torch.backends.cudnn.determinstic = True
torch.backends.cudnn.benchmark = False

In [ ]:
df_orig = pd.read_csv('train_data.csv', delimiter=",")
# print(df)
df = df_orig.copy()
df.shape

(4124, 17)

In [ ]:
df.head()

,SalePrice,YearBuilt,Size(sqf),Floor,HallwayType,HeatingType,AptManageType,N_Parkinglot(Ground),N_Parkinglot(Basement),TimeToBusStop,TimeToSubway,N_manager,N_elevators,SubwayStation,N_FacilitiesInApt,N_FacilitiesNearBy(Total),N_SchoolNearBy(Total)
0,141592,2006,814,3,terraced,individual_heating,management_in_trust,111.0,184.0,5min~10min,10min~15min,3.0,0.0,Kyungbuk_uni_hospital,5,6.0,9.0
1,51327,1985,587,8,corridor,individual_heating,self_management,80.0,76.0,0~5min,5min~10min,2.0,2.0,Daegu,3,12.0,4.0
2,48672,1985,587,6,corridor,individual_heating,self_management,80.0,76.0,0~5min,5min~10min,2.0,2.0,Daegu,3,12.0,4.0
3,380530,2006,2056,8,terraced,individual_heating,management_in_trust,249.0,536.0,0~5min,0-5min,5.0,11.0,Sin-nam,5,3.0,7.0
4,78318,1992,644,2,mixed,individual_heating,self_management,142.0,79.0,5min~10min,15min~20min,4.0,8.0,Myung-duk,3,9.0,14.0


In [ ]:
df.describe()

,SalePrice,YearBuilt,Size(sqf),Floor,N_Parkinglot(Ground),N_Parkinglot(Basement),N_manager,N_elevators,N_FacilitiesInApt,N_FacilitiesNearBy(Total),N_SchoolNearBy(Total)
count,4124.000000,4124.000000,4124.000000,4124.000000,4124.000000,4124.000000,4124.000000,4124.000000,4124.000000,4124.000000,4124.000000
mean,222177.477207,2002.977934,959.958778,11.994665,192.789040,572.857662,6.313773,11.055771,5.824200,9.860330,10.872696
std,106325.535526,8.765838,384.548456,7.581330,215.455916,408.179957,3.224556,7.717030,2.344331,3.444848,4.427445
min,34070.000000,1978.000000,135.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000
25%,144752.000000,1993.000000,644.000000,6.000000,11.000000,184.000000,5.000000,5.000000,4.000000,8.000000,7.000000
50%,209588.000000,2006.000000,910.000000,11.000000,100.000000,536.000000,6.000000,11.000000,5.000000,9.000000,10.000000
75%,291570.000000,2007.000000,1160.000000,17.000000,249.000000,798.000000,8.000000,16.000000,7.000000,13.000000,15.000000
max,585840.000000,2015.000000,2337.000000,43.000000,713.000000,1321.000000,14.000000,27.000000,10.000000,16.000000,17.000000


In [ ]:
df.HeatingType = (df.HeatingType=='individual_heating').astype(int)
df.AptManageType = (df.AptManageType=='management_in_trust').astype(int)

In [ ]:
categorical_columns = ["TimeToBusStop", "TimeToSubway", "SubwayStation", "HallwayType"]
categorical_values = pd.get_dummies(df[categorical_columns])
categorical_values.head()

,TimeToBusStop_0~5min,TimeToBusStop_10min~15min,TimeToBusStop_5min~10min,TimeToSubway_0-5min,TimeToSubway_10min~15min,TimeToSubway_15min~20min,TimeToSubway_5min~10min,TimeToSubway_no_bus_stop_nearby,SubwayStation_Bangoge,SubwayStation_Banwoldang,SubwayStation_Chil-sung-market,SubwayStation_Daegu,SubwayStation_Kyungbuk_uni_hospital,SubwayStation_Myung-duk,SubwayStation_Sin-nam,SubwayStation_no_subway_nearby,HallwayType_corridor,HallwayType_mixed,HallwayType_terraced
0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1
1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0
2,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0
3,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
4,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0


In [ ]:
df.drop(columns=categorical_columns,inplace=True)
df

,SalePrice,YearBuilt,Size(sqf),Floor,HeatingType,AptManageType,N_Parkinglot(Ground),N_Parkinglot(Basement),N_manager,N_elevators,N_FacilitiesInApt,N_FacilitiesNearBy(Total),N_SchoolNearBy(Total)
0,141592,2006,814,3,1,1,111.0,184.0,3.0,0.0,5,6.0,9.0
1,51327,1985,587,8,1,0,80.0,76.0,2.0,2.0,3,12.0,4.0
2,48672,1985,587,6,1,0,80.0,76.0,2.0,2.0,3,12.0,4.0
3,380530,2006,2056,8,1,1,249.0,536.0,5.0,11.0,5,3.0,7.0
4,78318,1992,644,2,1,0,142.0,79.0,4.0,8.0,3,9.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4119,570796,2007,1928,24,1,1,0.0,1270.0,14.0,16.0,10,9.0,10.0
4120,307079,2015,644,22,1,1,102.0,400.0,5.0,10.0,7,7.0,11.0
4121,357522,2007,868,20,1,1,0.0,1270.0,14.0,16.0,10,9.0,10.0
4122,312389,1978,1327,1,1,0,87.0,0.0,1.0,4.0,3,7.0,11.0


In [ ]:
train_indices = np.random.rand(len(df))>0.25
train_indices

array([ True,  True,  True, ...,  True,  True, False])

In [ ]:
numerical_data = torch.from_numpy(df.values[train_indices,1:]).float()
print(numerical_data)
categorical_data = torch.from_numpy(categorical_values.values[train_indices]).float()
print(categorical_data)
targets = torch.from_numpy(df.values[train_indices,0]).float()
print(targets)

test_numerical_data = torch.from_numpy(df.values[~train_indices,1:]).float()
test_categorical_data = torch.from_numpy(categorical_values.values[~train_indices]).float()
test_targets = torch.from_numpy(df.values[~train_indices,0]).float()

In [ ]:
train_dataset = data.TensorDataset(numerical_data,categorical_data,targets)
test_dataset = data.TensorDataset(test_numerical_data,test_categorical_data,test_targets)

In [ ]:
class Classifier_embeddings(nn.Module):
    def __init__(self, size_1, size_2, size_3, dropout=0.4):
        super().__init__()
        self.emb_layer = nn.Linear(categorical_data.shape[1], categorical_data.shape[1])
        self.act_emb = nn.Tanh()
        self.layer1 = nn.Linear(df.shape[1] -1 + categorical_data.shape[1], size_1)
        # self.bn1 = nn.BatchNorm1d(size_1)
        self.act_1 =  nn.LeakyReLU()
        self.d1 = nn.Dropout(dropout)
        self.layer2 = nn.Linear(size_1, size_2)
        # self.bn2 = nn.BatchNorm1d(size_2)
        self.act_2 =  nn.LeakyReLU()
        self.d2 = nn.Dropout(dropout)
        self.layer3 = nn.Linear(size_2, size_3)
        # self.bn3 = nn.BatchNorm1d(size_3)
        self.act_3 =  nn.LeakyReLU()
        self.d3 = nn.Dropout(dropout)
        self.layer4 = nn.Linear(size_3, 1)
    def forward(self, x, cat_x):
        cat_x_embedded = self.emb_layer(cat_x)
        cat_x_embedded = self.act_emb(cat_x_embedded)
        x = torch.cat([x,cat_x_embedded],dim=1)
        # x1 = self.layer1(x)
        # activation1 = self.act_1(self.bn1(x1))
        activation1 = self.act_1(self.layer1(x))
        activation1 = self.d1(activation1)
        # x2 = self.layer2(activation1)
        # activation2 = self.act_2(self.bn2(x2))
        activation2 = self.act_2(self.layer2(activation1))
        activation2 = self.d2(activation2)
        # x23 = self.layer3(activation2)
        # activation3 = self.act_3(self.bn2(x3))
        activation3 = self.act_3(self.layer3(activation2))
        activation3 = self.d3(activation3)
        output = self.layer4(activation3)
        return output

In [ ]:
def map_to_classes(output):
  classes = output.clone().detach()
  for idx, out in enumerate(output):
    if out <= 100000:
      classes[idx] = 0
    elif 100000 < out <= 350000:
      classes[idx] = 1
    elif out > 350000:
      classes[idx] = 2
  return classes

In [ ]:
def get_accuracy(model, data_loader):
    correct = 0
    total = 0
    model.eval() #*********#
    for x, cat_x, labels in data_loader:
        x, cat_x, labels = x.to(device), cat_x.to(device), labels.to(device)
        output = model(x, cat_x)

        output_classes = map_to_classes(output)
        labels_classes = map_to_classes(labels.view_as(output))

        correct += output_classes.eq(labels_classes).sum().item()
        total += x.shape[0]
    return correct / total

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False)
# criterion = nn.BCEWithLogitsLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.5)

In [ ]:
def train_and_calc_acc(model, num_epochs):
  criterion = nn.MSELoss()
  optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.02)

  iters = []
  losses = []
  train_acc = []
  val_acc = []
  for n in range(num_epochs):
      epoch_losses = []
      for x, cat_x, labels in iter(train_loader):
          x, cat_x, labels = x.to(device), cat_x.to(device), labels.to(device)
          model.train() 
          out = model(x, cat_x).squeeze()      
          loss = criterion(out, labels)
          loss.backward()  
          epoch_losses.append(loss.item())
          optimizer.step()              
          optimizer.zero_grad()

      if False:
        loss_mean = np.array(epoch_losses).mean()
        iters.append(n)
        losses.append(loss_mean)
        test_acc = get_accuracy(model, test_loader)
        print(f"Epoch {n} loss {loss_mean:.3} test_acc: {test_acc:.3}")
        train_acc.append(get_accuracy(model, train_loader)) # compute training accuracy 
        val_acc.append(test_acc)  # compute validation accuracy
            
  if False:
    print("Final Training Accuracy: {}".format(train_acc[-1]))
    print("Final Validation Accuracy: {}".format(val_acc[-1]))

  return get_accuracy(model, test_loader)

# model = Classifier_embeddings(3000, 4000, 5000, 0.4).to(device)
# train_and_calc_acc(model, 50)

# Przeszukiwanie parametrów

In [ ]:
dropout_values = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
for dropout_val in dropout_values:
  model = Classifier_embeddings(100, 200, 250, dropout_val).to(device)
  val_acc = train_and_calc_acc(model, num_epochs=150)
  print("dropout_value: ", dropout_val, ", val_accuracy:", val_acc)

In [ ]:
first_layer = [50, 100, 200, 400, 800, 1000, 2500, 5000]
second_layer = [50, 100, 200, 400, 800, 1000, 2500, 5000]
third_layer = [50, 100, 200, 400, 800, 1000, 2500, 5000]
results = pd.DataFrame()
for i in range(len(first_layer)):
    for j in range(len(second_layer)):
      for k in range(len(third_layer)):
        model = Classifier_embeddings(first_layer[i], second_layer[j], third_layer[k], dropout=0.4).to(device)
        val_acc = train_and_calc_acc(model, num_epochs=50)
        # print("first:", first_layer[i], ", second:", second_layer[j], ", third:", third_layer[k], ", val_accuracy:", val_acc)
        results = results.append({"first": first_layer[i], "second": second_layer[j], "third": third_layer[k], "acc": val_acc}, ignore_index=True)

In [ ]:
print(results)
results.to_csv("params_search.csv",index=False)
# results = pd.read_csv("params_search.csv")

In [ ]:
results.describe()

In [ ]:
results.idxmax()

In [ ]:
results.iloc[results["acc"].idxmax()]

In [ ]:
# finalny model
model = Classifier_embeddings(1000, 2000, 3000, 0.4).to(device)
train_and_calc_acc(model, 200)

0.8641618497109826

## Wyniki

In [ ]:
df_orig_test = pd.read_csv('test_data.csv', delimiter=",")
df_test = df_orig_test.copy()

df_test.HeatingType = (df_test.HeatingType=='individual_heating').astype(int)
df_test.AptManageType = (df_test.AptManageType=='management_in_trust').astype(int)
categorical_columns = ["TimeToBusStop", "TimeToSubway", "SubwayStation", "HallwayType"]
categorical_values = pd.get_dummies(df_test[categorical_columns])
df_test.drop(columns=categorical_columns,inplace=True)

numerical_data = torch.from_numpy(df_test.values).float()
categorical_data = torch.from_numpy(categorical_values.values).float()

eval_dataset = data.TensorDataset(numerical_data, categorical_data)
eval_data_loader = torch.utils.data.DataLoader(eval_dataset, batch_size=128, shuffle=False, drop_last=False)

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.02)

model.eval()

with torch.no_grad():
    for x, cat_x in eval_data_loader:

        x, cat_x = x.to(device), cat_x.to(device)
        preds = model(x, cat_x).squeeze()
        preds_classes = map_to_classes(preds)

        preds_np = preds_classes.numpy() #convert to Numpy array
        df = pd.DataFrame(preds_np) #convert to a dataframe
        df.to_csv("poniedzialek_grunwald_rozkosz.csv", mode='a', index=False, header=False) #save to file